# by

In today's session we'll use [transplants.dta](transplants.dta) to demonstrate a few useful commands that we will use quite frequently.

```Stata

use transplants, clear

```

What if we want a variable of the total number of records in each ABO blood type?

```Stata

bys abo: gen cat_n = _N
```

So what did this code achieve?

```Stata

tab cat_n
```

Let's add labels to the groups:

```Stata
#delimit ;
lab define abo_lab
   0 "0"
   1 "A"
   2 "B"
   3 "AB"
;
#delimit cr

lab values abo abo_lab
```

Now lets tabulate once again:

```Stata

tab cat_n
```

Ok. Do you remember these commands?

```Stata
di c(N)

di c(k)
```

Lets create a `disturbance` to our setup to learn a few additional commands and their value to us:

```Stata

qui do https://raw.githubusercontent.com/jhustata/book/main/sample-keep.ado
```

Is that a new command? Or are you already familiar with it? How about this:

```Stata
samplekeep
```


